In [307]:
import pandas as pd
import numpy as np
import random

In [308]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Taking a random sample from the dataset

In [372]:
# the random sample is skipping 999 rows and taking 1 row on average 
dy = pd.read_csv(
         '../../../JanOct18_trimmed.csv')#, skiprows = lambda i: i>0 and random.random() > 0.1)

# dy16 = pd.read_csv(
#          '../../../JanOct16_trimmed.csv', skiprows = lambda i: i>0 and random.random() > 0.001)

# dy17 = pd.read_csv(
#          '../../../JanOct17_trimmed.csv', skiprows = lambda i: i>0 and random.random() > 0.001)

# dy18 = pd.read_csv(
#          '../../../JanOct18_trimmed.csv', skiprows = lambda i: i>0 and random.random() > 0.001)


In [310]:
dn = pd.read_csv(
         '../../../Nov17_trimmed.csv')
#,         skiprows=lambda i: i>0 and random.random() > 0.1)

In [373]:
vehicles = dy.drop_duplicates(subset = 'VEHICLE_ID',keep='first')

In [312]:
targets = dn[dn['PROD_GROUP_CODE'] == 5]['INDIV_ID'].unique()


In [374]:
year_indivs = pd.DataFrame({'indiv':dy['INDIV_ID'].unique()})

In [314]:
year_indivs['label'] = np.where(year_indivs['indiv'].isin(targets), 1, 0) 

In [376]:
year_indivs['cust_in_top_10_sales'] = np.where(year_indivs['indiv'].isin(dy.INDIV_ID.value_counts().nlargest(int(len(year_indivs)/10))),1,0)

In [377]:
year_indivs.label.value_counts()

AttributeError: 'DataFrame' object has no attribute 'label'

In [378]:
len(year_indivs)

2624108

In [ ]:
# def label_training__data(year_file_name, nov_file_name):
#     year_dat = pd.read_csv(year_file_name)
#     nov_dat = nov_file_name

### Feature : is the store among the top performers regarding total number of transactions in general?

In [379]:
top_stores_trans_list = dy.STORE_ID.value_counts().nlargest(int(len(dy)/100)).index

In [380]:
len(top_stores_trans_list)

2243

get individuals visiting those stores

In [381]:
indiv_store_list = dy[dy['STORE_ID'].isin(top_stores_trans_list)]['INDIV_ID']

In [382]:
year_indivs['store_top_tran'] = np.where(year_indivs.indiv.isin(indiv_store_list), 1, 0)

In [383]:
# sanity check
year_indivs['store_top_tran'].value_counts()

1    2624108
Name: store_top_tran, dtype: int64

### Feature : is the store among the top performers regarding total number of TIRES transactions ?

In [384]:
tires = dy.query('PROD_GROUP_CODE == 5')

In [385]:
top_stores_tires_list = tires.STORE_ID.value_counts().nlargest(int(len(dy)/100)).index

In [386]:
top_stores_tires_list

Int64Index([237119, 240187, 517909, 326506, 517895, 517658, 323022, 517585,
            517720,  27871,
            ...
            793846, 793855, 794381, 793848, 793849, 793850, 793863, 793845,
            793847, 793853],
           dtype='int64', length=2240)

get individuals visiting those stores

In [387]:
indiv_store_tires_list = dy[dy['STORE_ID'].isin(top_stores_tires_list)]['INDIV_ID']

In [388]:
year_indivs['store_top_tires_tran'] = np.where(year_indivs.indiv.isin(indiv_store_tires_list), 1, 0)

In [389]:
# sanity check
year_indivs['store_top_tires_tran'].value_counts()

1    2624106
0          2
Name: store_top_tires_tran, dtype: int64

In [390]:
year_indivs.head()

,indiv,cust_in_top_10_sales,store_top_tran,store_top_tires_tran
0,587295308.0,0,1,1
1,289068522.0,0,1,1
2,267651496.0,0,1,1
3,272771622.0,0,1,1
4,286864796.0,0,1,1


In [391]:
indiv_store_tires_list = dy[dy['STORE_ID'].isin(top_stores_tires_list)]['INDIV_ID']

In [392]:
year_indivs['store_top_tires_tran'] = np.where(year_indivs.indiv.isin(indiv_store_tires_list), 1, 0)

In [393]:
# sanity check
year_indivs['store_top_tires_tran'].value_counts()

1    2624106
0          2
Name: store_top_tires_tran, dtype: int64

## Feature: has this vehicle recently had a tire service? 

consider adding a certain time limit 

### list of vehicles with recent tires' services 

In [394]:
vhc_tire_svc_array = dy.query('CATEGORY_DESC == "Tire Services"')['VEHICLE_ID'].unique()

In [395]:
indiv_vhc_svc_list = dy[dy['VEHICLE_ID'].isin(vhc_tire_svc_array)]['INDIV_ID']

In [396]:
year_indivs['vehicle_tire_svc'] = np.where(year_indivs.indiv.isin(indiv_vhc_svc_list), 1, 0)

In [397]:
# sanity check
year_indivs['vehicle_tire_svc'].value_counts()

1    1814868
0     809240
Name: vehicle_tire_svc, dtype: int64

## Feature: has anyone purchased a tire for this vehicle before?

we need to settle on the definition of tire purchase, is it 5 only or after removing others as well?

In [398]:
vhc_tire_pur_array = dy.query('PROD_GROUP_DESC == "Tires" & SEGMENT_DESC != "Others"')['VEHICLE_ID'].unique()

In [399]:
vhc_tire_pur_array

array([913599587, 934945887, 954343100, ..., 978160104, 971530430,
       971130506])

In [400]:
indiv_vhc_pur_list = dy[dy['VEHICLE_ID'].isin(vhc_tire_pur_array)]['INDIV_ID']

In [401]:
year_indivs['vhc_tire_purchase'] = np.where(year_indivs.indiv.isin(indiv_vhc_pur_list), 1, 0)

In [402]:
# sanity check
year_indivs['vhc_tire_purchase'].value_counts()

0    1569505
1    1054603
Name: vhc_tire_purchase, dtype: int64

## Feature : when was the last time this vehicle bought a new tire? [last year - 2 years - 3 years]

In [403]:
dy['DATE'] = pd.to_datetime(dy['DATE'])

In [404]:
dy['year'] = pd.DatetimeIndex(dy['DATE']).year
dy['month'] = pd.DatetimeIndex(dy['DATE']).month

In [405]:
dy_vhc_tire_purch = dy.query('PROD_GROUP_DESC == "Tires" & SEGMENT_DESC != "Others"')

In [406]:
month_look = dy_vhc_tire_purch.groupby('VEHICLE_ID').agg({'month':['max'] })

Max is the latest month the tire was purchased. If the last month to purchase a tire was before 6, the vehicle may need to replace tires. 
Of course, this needs a change, but after we combine datasets of different years. 

In [407]:
month_look.columns = ['last_mo_tire_purch']

In [408]:
vhc_early_purch_tires = month_look.reset_index().query('last_mo_tire_purch < 6')['VEHICLE_ID']

In [409]:
indiv_vhc_early_pur_list = dy[dy['VEHICLE_ID'].isin(vhc_early_purch_tires)]['INDIV_ID']

In [410]:
year_indivs['vhc_early_tire_purchase'] = np.where(year_indivs.indiv.isin(indiv_vhc_early_pur_list), 1, 0)

In [411]:
# sanity check
year_indivs['vhc_early_tire_purchase'].value_counts()

0    2142703
1     481405
Name: vhc_early_tire_purchase, dtype: int64

# Alert 

too many vehicles with ID =1 

### Here I need to understand what to do with units 

## Feature: top customers regarding overall transactions  

In [412]:
top_cust_overall_list = dy.INDIV_ID.value_counts().nlargest(int(len(dy)/10))

In [413]:
year_indivs['top_cust_overall'] = np.where(year_indivs['indiv'].isin(top_cust_overall_list), 1, 0)

## Feature: top customers regarding tires' purchase 

In [414]:
top_cust_tire_purch_array = dy_vhc_tire_purch.INDIV_ID.value_counts().nlargest(int(len(dy_vhc_tire_purch)/10)).index

In [415]:
year_indivs['top_cust_tire_purch'] = np.where(year_indivs['indiv'].isin(top_cust_tire_purch_array), 1, 0)

In [416]:
year_indivs['top_cust_tire_purch'].value_counts()

0    2251724
1     372384
Name: top_cust_tire_purch, dtype: int64

In [417]:
year_indivs.head(10)

,indiv,cust_in_top_10_sales,store_top_tran,store_top_tires_tran,vehicle_tire_svc,vhc_tire_purchase,vhc_early_tire_purchase,top_cust_overall,top_cust_tire_purch
0,587295308.0,0,1,1,0,0,0,0,0
1,289068522.0,0,1,1,1,0,0,0,0
2,267651496.0,0,1,1,1,1,1,0,0
3,272771622.0,0,1,1,1,1,1,0,1
4,286864796.0,0,1,1,1,1,0,0,0
5,612149572.0,0,1,1,1,0,0,0,0
6,289878575.0,0,1,1,1,1,0,0,1
7,316506474.0,0,1,1,1,1,1,0,1
8,292361182.0,0,1,1,1,1,0,0,1
9,289502694.0,0,1,1,1,1,1,0,1


In [418]:
dy.head(10)

,Unnamed: 0,Unnamed: 0.1,STORE_ID,TRAN_ID,DATE,ARTICLE_ID,INDIV_ID,VEHICLE_ID,UNITS,SALES,STATE_CODE,ZIP_CODE,MSA,MAKE,MODEL,SUB_MODEL,MODEL_YEAR,PROD_GROUP_CODE,PROD_GROUP_DESC,CATEGORY_CODE,CATEGORY_DESC,SEGMENT_CODE,SEGMENT_DESC,CLASS_CODE,CLASS_DESC,DISCOUNT_FLAG,CROSS_SECTION,ASPECT_RATIO,RIM_SIZE,EMAIL_OPTIN_IND,AH1_RES_BUS_INDC,SUPP1_BUS_PANDER,year,month
0,10,10,27081,992088410,2018-01-25,7005180,587295308.0,964475905,0.0,-30.00,NY,13039,"SYRACUSE,N",CHEVROLET,MALIBU LTZ,LTZ 1LTZ Pkg.,2013.0,4.0,Services,83.0,Steering/Suspension/Drivetrain,96.0,Alignments,190.0,Lifetime Alignment,Y,NaN,NaN,NaN,Y,R,N,2018,1
1,11,11,27081,992088410,2018-01-25,7005229,587295308.0,964475905,0.0,199.99,NY,13039,"SYRACUSE,N",CHEVROLET,MALIBU LTZ,LTZ 1LTZ Pkg.,2013.0,4.0,Services,83.0,Steering/Suspension/Drivetrain,96.0,Alignments,190.0,Lifetime Alignment,N,NaN,NaN,NaN,Y,R,N,2018,1
2,12,12,27081,992088410,2018-01-25,7005219,587295308.0,964475905,0.0,0.00,NY,13039,"SYRACUSE,N",CHEVROLET,MALIBU LTZ,LTZ 1LTZ Pkg.,2013.0,4.0,Services,83.0,Steering/Suspension/Drivetrain,96.0,Alignments,190.0,Lifetime Alignment,Y,NaN,NaN,NaN,Y,R,N,2018,1
3,27,27,18929,992021540,2018-01-02,7028789,289068522.0,964961935,0.0,21.99,AL,35805,HUNTSVILLE,JEEP,PATRIOT SPORT,Sport,2013.0,4.0,Services,81.0,Miscellaneous Services,109.0,Inspection Services,475.0,Complete Vehicle Inspection,N,NaN,NaN,NaN,Y,R,N,2018,1
4,30,30,817,992441300,2018-01-08,7015016,267651496.0,913599587,0.0,0.00,MA,1420,"WORCESTER,",HYUNDAI,SONATA SE,SE V6,2009.0,4.0,Services,62.0,Tire Services,91.0,Tire Services,139.0,Tire Mounting,N,NaN,NaN,NaN,Y,R,N,2018,1
5,31,31,817,992441300,2018-01-08,7013632,267651496.0,913599587,0.0,12.99,MA,1420,"WORCESTER,",HYUNDAI,SONATA SE,SE V6,2009.0,4.0,Services,62.0,Tire Services,91.0,Tire Services,147.0,Balance,N,NaN,NaN,NaN,Y,R,N,2018,1
6,32,32,817,992441300,2018-01-08,3438,267651496.0,913599587,1.0,60.99,MA,1420,"WORCESTER,",HYUNDAI,SONATA SE,SE V6,2009.0,5.0,Tires,26.0,Passenger Tires,27.0,Touring,33.0,Mass Market Tires,N,215,55,17,Y,R,N,2018,1
7,33,33,817,992441300,2018-01-08,7097782,267651496.0,913599587,0.0,8.84,MA,1420,"WORCESTER,",HYUNDAI,SONATA SE,SE V6,2009.0,5.0,Tires,20158.0,Road Hazard,20159.0,Road Hazard,20160.0,Road Hazard,N,NONE,NONE,NONE,Y,R,N,2018,1
8,58,58,781219,990091660,2018-01-04,7001123,272771622.0,934945887,0.0,0.00,FL,34233,"SARASOTA,F",HYUNDAI,SONATA GLS,GLS,2014.0,4.0,Services,62.0,Tire Services,91.0,Tire Services,155.0,Misc. Tire Service,N,NaN,NaN,NaN,Y,R,N,2018,1
9,59,59,781219,990091660,2018-01-04,7008190,272771622.0,934945887,0.0,2.99,FL,34233,"SARASOTA,F",HYUNDAI,SONATA GLS,GLS,2014.0,4.0,Services,62.0,Tire Services,91.0,Tire Services,62150.0,Tire Pressure Monitoring Systems,N,NaN,NaN,NaN,Y,R,N,2018,1


Test

In [419]:
top_make_tire_purch_array = dy_vhc_tire_purch.MAKE.value_counts().nlargest(int(len(dy_vhc_tire_purch)/20)).index.tolist()[0:10]

In [420]:
top_make_tire_purch_array

['TOYOTA',
 'FORD',
 'HONDA',
 'CHEVROLET',
 'NISSAN',
 'HYUNDAI',
 'DODGE',
 'KIA',
 'JEEP',
 'CHRYSLER']

In [421]:
dy['MAKE'] = dy['MAKE'].astype('str')

In [422]:
indiv_make_tire_purch = dy[dy['MAKE'].isin(top_make_tire_purch_array)]['INDIV_ID']

In [423]:
year_indivs['indiv_make_tire_pch'] = np.where(year_indivs.indiv.isin(indiv_make_tire_purch), 1, 0)

In [424]:
year_indivs['indiv_make_tire_pch'].value_counts()

1    2007178
0     616930
Name: indiv_make_tire_pch, dtype: int64

In [425]:
year_indivs.head(10)

,indiv,cust_in_top_10_sales,store_top_tran,store_top_tires_tran,vehicle_tire_svc,vhc_tire_purchase,vhc_early_tire_purchase,top_cust_overall,top_cust_tire_purch,indiv_make_tire_pch
0,587295308.0,0,1,1,0,0,0,0,0,1
1,289068522.0,0,1,1,1,0,0,0,0,1
2,267651496.0,0,1,1,1,1,1,0,0,1
3,272771622.0,0,1,1,1,1,1,0,1,1
4,286864796.0,0,1,1,1,1,0,0,0,0
5,612149572.0,0,1,1,1,0,0,0,0,1
6,289878575.0,0,1,1,1,1,0,0,1,1
7,316506474.0,0,1,1,1,1,1,0,1,1
8,292361182.0,0,1,1,1,1,0,0,1,0
9,289502694.0,0,1,1,1,1,1,0,1,1


In [426]:
dy['tire_purchase'] = np.where((dy['PROD_GROUP_DESC'] == "Tires") & (dy['SEGMENT_DESC'] != "Others"), 1, 0)

df2 = dy.groupby(['MAKE', 'tire_purchase']).agg({'tire_purchase': 'count'})
df3 = df2.groupby(level=0).apply(lambda x:100 * x / float(x.sum()))
df3.columns = ['total'] # I need to rename the column to handle multiindex caused by groupby 

top_make_tires_purch_prop_array = df3.reset_index().query('tire_purchase == 1').sort_values(by = 'total', ascending = False).nlargest(int(len(df3)/10),'total')['MAKE'].tolist()[0:1000]
                                                                                                                                      
indiv_make_tire_purch_prop = dy[dy['MAKE'].isin(top_make_tires_purch_prop_array)]['INDIV_ID']

year_indivs['make_prop_tire'] = np.where(year_indivs.indiv.isin(indiv_make_tire_purch_prop), 1, 0)

year_indivs['make_prop_tire'].value_counts()

0    2622630
1       1478
Name: make_prop_tire, dtype: int64

In [427]:
df2 = dy.groupby(['MODEL', 'tire_purchase']).agg({'tire_purchase': 'count'})
df3 = df2.groupby(level=0).apply(lambda x:100 * x / float(x.sum()))
df3.columns = ['total'] # I need to rename the column to handle multiindex caused by groupby 

dy['MODEL'] = dy['MODEL'].astype('str')

top_MODEL_tires_purch_prop_array = df3.reset_index().query('tire_purchase == 1').sort_values(by = 'total', ascending = False).nlargest(int(len(df3)/10),'total')['MODEL'].tolist()[0:1000]

indiv_MODEL_tire_purch_prop = dy[dy['MODEL'].isin(top_MODEL_tires_purch_prop_array)]['INDIV_ID']
                                                                                                                                       
year_indivs['MODEL_prop_tire'] = np.where(year_indivs.indiv.isin(indiv_MODEL_tire_purch_prop), 1, 0)
                                                                                                                                       
year_indivs['MODEL_prop_tire'].value_counts()                                                                                                                                    

0    2616701
1       7407
Name: MODEL_prop_tire, dtype: int64

In [428]:
df2 = dy.groupby(['SUB_MODEL', 'tire_purchase']).agg({'tire_purchase': 'count'})
df3 = df2.groupby(level=0).apply(lambda x:100 * x / float(x.sum()))
df3.columns = ['total'] # I need to rename the column to handle multiindex caused by groupby

dy['SUB_MODEL'] = dy['SUB_MODEL'].astype('str')

top_SUB_MODEL_tires_purch_prop_array = df3.reset_index().query('tire_purchase == 1').sort_values(by = 'total', ascending = False).nlargest(int(len(df3)/10), 'total')['SUB_MODEL'].tolist()[0:1000]            

indiv_SUB_MODEL_tire_purch_prop = dy[dy['SUB_MODEL'].isin(top_SUB_MODEL_tires_purch_prop_array)]['INDIV_ID']
                                                                                                                                               
year_indivs['SUB_MODEL_prop_tire'] = np.where(year_indivs.indiv.isin(indiv_SUB_MODEL_tire_purch_prop), 1, 0)
                                                                                                                                               
year_indivs['SUB_MODEL_prop_tire'].value_counts()

0    2605490
1      18618
Name: SUB_MODEL_prop_tire, dtype: int64

In [ ]:
year_indivs.head(10)

,indiv,cust_in_top_10_sales,store_top_tran,store_top_tires_tran,vehicle_tire_svc,vhc_tire_purchase,vhc_early_tire_purchase,top_cust_overall,top_cust_tire_purch,indiv_make_tire_pch,make_prop_tire,MODEL_prop_tire,SUB_MODEL_prop_tire
0,587295308.0,0,1,1,0,0,0,0,0,1,0,0,0
1,289068522.0,0,1,1,1,0,0,0,0,1,0,0,0
2,267651496.0,0,1,1,1,1,1,0,0,1,0,0,0
3,272771622.0,0,1,1,1,1,1,0,1,1,0,0,0
4,286864796.0,0,1,1,1,1,0,0,0,0,0,0,0
5,612149572.0,0,1,1,1,0,0,0,0,1,0,0,0
6,289878575.0,0,1,1,1,1,0,0,1,1,0,0,0
7,316506474.0,0,1,1,1,1,1,0,1,1,0,0,0
8,292361182.0,0,1,1,1,1,0,0,1,0,0,0,0
9,289502694.0,0,1,1,1,1,1,0,1,1,0,0,0


In [ ]:
year_indivs.to_csv('individuals18.csv')

In [ ]:
# 